In [ ]:
from pygwan import WhatsApp

messenger = WhatsApp("EAAkWyEKmb90BOx04XQwZBA2hgghGR8Q6MX8ELwLWodPj9wfSRZASegj4Mr1rT0v0OfBH9EPxnAjQJTDkse5PR1EJh0YxEFWmtIKtn1Nwigrqxaa7ljN9PZBX0TOE3cPH7t1OD6ljYValWE3z0b7m9ylV3dQNSG0MTZAkO6jiCj56cTT5InAx85jMhmMcnUtu0DrG2Afl69NNc8120SufKUIS1aBOqLPo6ytfhnmNvgkh6sJJte4ZD","644904198709959" )
# Initialize the WhatsApp messenger with your access token and phone number ID
# Note: Replace the access token and phone number ID with your own values
# Send a message to all contacts

# Send a message to a specific contact
messenger.send_message(
    "Hello, this is a test message from Nyasha's bot ! Confirm if you received this message",
    recipient_id="263774694160"  # Replace with the recipient's phone number in international format
 ) 

 




{'messaging_product': 'whatsapp',
 'contacts': [{'input': '263774694160', 'wa_id': '263774694160'}],
 'messages': [{'id': 'wamid.HBgMMjYzNzc0Njk0MTYwFQIAERgSNjU1NzUwMTg5QTEyRjVBQ0EzAA=='}]}

In [12]:
from flask import Flask, request
from dotenv import load_dotenv
import os
from pygwan import WhatsApp

load_dotenv()

app = Flask(__name__)

whatsapp = WhatsApp(
    token=os.getenv("WHATSAPP_TOKEN"),
    phone_number_id=os.getenv("PHONE_NUMBER_ID")
)

sessions = {}

donation_types = ["Monthly Contributions", "August Conference", "Youth Conference"]

@app.route("/webhook", methods=["GET", "POST"])
def webhook():
    if request.method == "GET":
        if request.args.get("hub.verify_token") == os.getenv("VERIFY_TOKEN"):
            return request.args.get("hub.challenge")
        return "Invalid verify token", 403

    data = request.get_json()
    if whatsapp.is_message(data):
        phone = whatsapp.get_mobile(data)
        name = whatsapp.get_name(data)
        msg = whatsapp.get_message(data)

        if phone not in sessions:
            sessions[phone] = {"step": "name", "data": {}}
            whatsapp.send_message(f"Hi {name}! 🙏🏾 Welcome to the Church Donation Bot. What’s your *full name*?", phone)
            return "ok"

        session = sessions[phone]

        if session["step"] == "name":
            session["data"]["name"] = msg
            session["step"] = "amount"
            whatsapp.send_message("💰 How much would you like to donate (e.g. 5000)?", phone)

        elif session["step"] == "amount":
            session["data"]["amount"] = msg
            session["step"] = "donation_type"
            buttons = {
                "header": "Choose donation type",
                "body": "Please select the purpose of your donation:",
                "footer": "Church Giving",
                "action": {
                    "buttons": [{"id": str(i), "title": option[:20]} for i, option in enumerate(donation_types)]
                }
            }
            whatsapp.send_button(buttons, phone)

        elif session["step"] == "donation_type":
            session["data"]["donation_type"] = msg
            session["step"] = "region"
            whatsapp.send_message("🌍 What is your region or location?", phone)

        elif session["step"] == "region":
            session["data"]["region"] = msg
            session["step"] = "note"
            whatsapp.send_message("📝 Any extra notes for the finance director?", phone)

        elif session["step"] == "note":
            session["data"]["note"] = msg
            session["step"] = "done"
            summary = session["data"]
            confirm_message = (
                f"✅ Thank you {summary['name']}!\n\n"
                f"💰 Amount: {summary['amount']}\n"
                f"📌 Type: {summary['donation_type']}\n"
                f"🌍 Region: {summary['region']}\n"
                f"📝 Note: {summary['note']}\n\n"
                "We will now send a payment link and notify the finance director after payment is complete."
            )
            whatsapp.send_message(confirm_message, phone)

            # TODO: Trigger Paynow here
            notify_finance_director(summary)
            del sessions[phone]

    return "ok"

def notify_finance_director(d):
    finance_phone = os.getenv("FINANCE_PHONE")  # Must be set in .env
    msg = (
        f"📥 *New Church Donation!*\n\n"
        f"🙍🏽 Name: {d['name']}\n"
        f"💵 Amount: {d['amount']}\n"
        f"📌 Purpose: {d['donation_type']}\n"
        f"🌍 Region: {d['region']}\n"
        f"📝 Note: {d['note']}"
    )
    whatsapp.send_message(msg, finance_phone)

if __name__ == "__main__":
    app.run(port=5000, debug=True)





 * Serving Flask app '__main__'


 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


SystemExit: 1